In [1]:
# In this notebook we will work off regression_model3 and change all negative delays to zero
# We will also round all the values of arr_delay to the nearest 5 minutes and try to predict 
# The delay to the nearest 5 minutes 

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import scipy.stats as stats
import numpy as np

import plotly.express as px
import plotly.graph_objs as go
from sklearn.preprocessing import MinMaxScaler

from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

In [4]:
# Imported this dataframe that was created in our EDA notebook
flights_df = pd.read_csv("/Users/ckz/Desktop/DATA_SCIENCE_BOOTCAMP/Midterm-project/flights_df.csv")

# print the first few rows of the DataFrame
flights_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,first_dep_time,total_add_gtime,longest_add_gtime,no_name,month,dep_hour,arr_hour,taxi_time,origin_state,flight_distance_category
0,2018-01-01,B6,B6,B6,880,B6,N794JB,880,12954,LGB,...,NaN,NaN,NaN,NaN,1,7,8,83.0,CA,SHORT
1,2018-01-01,B6,B6,B6,885,B6,N337JB,885,12478,JFK,...,NaN,NaN,NaN,NaN,1,7,9,120.0,NY,SHORT
2,2018-01-01,B6,B6,B6,886,B6,N337JB,886,14492,RDU,...,NaN,NaN,NaN,NaN,1,10,11,102.0,NC,SHORT
3,2018-01-01,B6,B6,B6,889,B6,N318JB,889,11278,DCA,...,NaN,NaN,NaN,NaN,1,18,19,162.0,DC,SHORT
4,2018-01-01,B6,B6,B6,891,B6,N348JB,891,10721,BOS,...,NaN,NaN,NaN,NaN,1,6,9,317.0,MA,MEDIUM


In [6]:

# Replace negative values with 0
flights_df['arr_delay'] = np.where(flights_df['arr_delay'] < 0, 0, flights_df['arr_delay'])

# Round non-negative values to the nearest 5-minute mark
flights_df['arr_delay'] = flights_df['arr_delay'].apply(lambda x: int(round(x / 5)) * 5)

In [7]:
flights_df['avg_monthly_arr_delay'] = flights_df.groupby('month')['arr_delay'].transform('mean')

In [8]:
# This feature creates the avg taxi times for the arrival hour 
flights_df['avg_taxi_times_arr'] = flights_df.groupby('arr_hour')['taxi_time'].transform('mean')

In [9]:
# Group the flights data by origin airport and count the number of flights for each airport
airport_counts = flights_df.groupby('origin')['flights'].transform('count')

# Determine the thresholds for each category based on the quartiles
thresholds = [0, airport_counts.quantile(0.25), airport_counts.quantile(0.5), airport_counts.quantile(0.75), np.inf]

# Define the categories
categories = ['Not Busy', 'Moderate', 'Busy', 'Very Busy']

# Assign the traffic category based on the number of flights from each airport
flights_df['traffic'] = pd.cut(airport_counts, bins=thresholds, labels=categories, right=False)

In [10]:
from pandas.tseries.holiday import USFederalHolidayCalendar as holiday_calendar
from datetime import datetime

In [11]:
flights_df['fl_date'] = flights_df['fl_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

In [12]:
# holiday_cal
cal = holiday_calendar()
dr = flights_df['fl_date'] 
holidays = cal.holidays(start=dr.min(), end=dr.max())
flights_df['holiday'] = flights_df['fl_date'].dt.date.astype('datetime64').isin(holidays).astype(int)
flights_df.tail(5)

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,month,dep_hour,arr_hour,taxi_time,origin_state,flight_distance_category,avg_monthly_arr_delay,avg_taxi_times_arr,traffic,holiday
1559040,2019-07-31,DL,DL,DL,2899,DL,N932AT,2899,10397,ATL,...,7,15,17,114.0,GA,SHORT,17.041748,177.955554,Very Busy,0
1559041,2019-07-31,DL,DL,DL,2899,DL,N932AT,2899,11057,CLT,...,7,17,18,83.0,NC,SHORT,17.041748,170.038468,Very Busy,0
1559042,2019-07-31,DL,DL,DL,2900,DL,N301DQ,2900,10397,ATL,...,7,12,14,175.0,GA,MEDIUM,17.041748,178.305499,Very Busy,0
1559043,2019-07-31,DL,DL,DL,2900,DL,N301DQ,2900,11624,EYW,...,7,15,17,126.0,FL,MEDIUM,17.041748,177.955554,Not Busy,0
1559044,2019-07-31,DL,DL,DL,2901,DL,N397DA,2901,11292,DEN,...,7,7,9,159.0,CO,SHORT,17.041748,146.300733,Very Busy,0


In [13]:
flights_df.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name', 'month', 'dep_hour', 'arr_hour',
       'taxi_time', 'origin_state', 'flight_distance_category',
       'avg_monthly_arr_delay', 'avg_taxi_times_arr', 'traffic', 'holiday'],
      dtype='object')

In [14]:
flights_df['fl_date'] = pd.to_datetime(flights_df['fl_date'])

flights_df['day_of_week'] = flights_df['fl_date'].dt.dayofweek

day_mapping = {0: 'Monday', 1: 'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
flights_df['day_of_week'] = flights_df['day_of_week'].map(day_mapping)

In [15]:
flights_df['avg_hourly_arr_delay'] = flights_df.groupby('arr_hour')['arr_delay'].transform('mean')

In [16]:
flights_df['avg_mktcarrier_arr_delay'] = flights_df.groupby('mkt_unique_carrier')['arr_delay'].transform('mean')

In [17]:
flights_df['avg_airport_arr_delay'] = flights_df.groupby('origin_airport_id')['arr_delay'].transform('mean')

In [18]:
flights_df['avg_plane_arr_delay'] = flights_df.groupby('tail_num')['arr_delay'].transform('mean')

In [19]:
flights_df['avg_destcity_arr_delay'] = flights_df.groupby('dest_city_name')['arr_delay'].transform('mean')

In [20]:
flights_df['avg_state_arr_delay'] = flights_df.groupby('origin_state')['arr_delay'].transform('mean')

In [21]:
flights_df['avg_dow_arr_delay'] = flights_df.groupby('day_of_week')['arr_delay'].transform('mean')

In [22]:
flights_df['avg_dow_arr_delay'] = flights_df.groupby('day_of_week')['arr_delay'].transform('mean')

In [23]:
# # Shuffle the data in the DataFrame and create a sample with half the size
sample_size = len(flights_df) // 10

flights_df_sample = flights_df.sample(n=sample_size, random_state=42)

## Reset the index of the shuffled sample

flights_df_sample.reset_index(drop=True, inplace=True)

In [24]:
flights_df_sample.columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time', 'dep_time',
       'dep_delay', 'taxi_out', 'wheels_off', 'wheels_on', 'taxi_in',
       'crs_arr_time', 'arr_time', 'arr_delay', 'cancelled',
       'cancellation_code', 'diverted', 'dup', 'crs_elapsed_time',
       'actual_elapsed_time', 'air_time', 'flights', 'distance',
       'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay',
       'late_aircraft_delay', 'first_dep_time', 'total_add_gtime',
       'longest_add_gtime', 'no_name', 'month', 'dep_hour', 'arr_hour',
       'taxi_time', 'origin_state', 'flight_distance_category',
       'avg_monthly_arr_delay', 'avg_taxi_times_arr', 'traffic', 'holiday',
       'day_of_week', 'avg_hourly_arr_delay', 'avg_mktcarrier_arr_delay',
       'avg_ai

In [25]:
flights_df_sample = flights_df_sample[['fl_date','mkt_unique_carrier','arr_delay',
        'cancelled', 'diverted',
        'air_time','distance',
        'month', 'dep_hour', 'arr_hour',
       'origin_state', 'flight_distance_category',
       'avg_monthly_arr_delay', 'avg_taxi_times_arr', 'traffic', 'holiday','avg_hourly_arr_delay', 'avg_mktcarrier_arr_delay',
       'avg_airport_arr_delay', 'avg_plane_arr_delay',
       'avg_destcity_arr_delay', 'avg_state_arr_delay','day_of_week','avg_dow_arr_delay']]

In [26]:
numerical_columns = ['air_time','distance','avg_taxi_times_arr','avg_hourly_arr_delay', 'avg_mktcarrier_arr_delay', 'avg_airport_arr_delay', 'avg_plane_arr_delay', 'avg_destcity_arr_delay', 'avg_state_arr_delay','avg_dow_arr_delay']
                    
categorical_columns = ['mkt_unique_carrier','diverted','month','dep_hour','arr_hour','origin_state','flight_distance_category','traffic','holiday','day_of_week']




In [27]:
df_encoded = pd.get_dummies(data=flights_df_sample, columns= categorical_columns , drop_first=False)

In [28]:
df_encoded = df_encoded.drop(['fl_date'], axis=1)

In [29]:
df_encoded = df_encoded.drop(['arr_delay'], axis=1)

In [30]:
# Linear regression model

# Separate the input features (X) and the target variable (y)
X = df_encoded

y = flights_df_sample['arr_delay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
score = model.score(X_test, y_test)

# Print the coefficient and intercept of the linear regression model
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('R^2 Score:', score)

Coefficients: [ 1.65647758e-15  3.10058440e-01 -3.68830861e-02  1.00305305e+00
  7.47827111e-04  3.90750071e-01 -2.11018943e-01  7.22690115e-01
  9.91691336e-01  8.79671565e-01 -4.78907162e-01  6.05732667e-01
 -7.32300701e-01 -1.71024166e+00 -1.21592992e+00  1.79267198e+00
  1.29569739e+00 -7.19703466e-01 -1.23086515e+00  5.89560938e-01
 -2.37105299e+00  3.44086950e+00  8.61294076e-01  4.08562073e-14
 -4.25440698e-01 -1.04995126e+00  9.84900611e-02  6.51778131e-01
 -2.61038298e-01  6.56845441e-01  6.69044373e-02  3.93048764e-01
  2.08344832e-01 -4.81683662e-02 -1.26629357e-01 -1.64183681e-01
  5.05428279e-01 -4.33216823e+00  5.78826902e+00 -6.06945714e+00
 -3.50898660e+00 -4.38220820e+00 -3.38334255e+00 -2.61314523e+00
  7.91257697e-02  1.14578760e-01  8.07127679e-01  3.78590497e-01
  4.03825357e-01  8.29193990e-01  2.22008800e-01 -2.91801396e-01
  6.06498157e-01  1.92537840e+00  1.25456595e+00  1.29340220e+00
  2.26885702e+00 -2.05727182e-01  4.58396345e-01  7.85159030e+00
  3.1547003

In [31]:
### linear regression doesnt seem to work with this arr_delay therefore we will try a different model like random forests.

In [32]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Separate the input features (X) and the target variable (y)
X = df_encoded
y = flights_df_sample['arr_delay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Random Forest regressor model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model using the training data
rf_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = rf_model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse)
print('R^2 Score:', r2)

Mean Squared Error: 1955.6976159256565
R^2 Score: -0.03689057179800659


In [33]:
df_encoded.columns

Index(['cancelled', 'air_time', 'distance', 'avg_monthly_arr_delay',
       'avg_taxi_times_arr', 'avg_hourly_arr_delay',
       'avg_mktcarrier_arr_delay', 'avg_airport_arr_delay',
       'avg_plane_arr_delay', 'avg_destcity_arr_delay',
       ...
       'traffic_Very Busy', 'holiday_0', 'holiday_1', 'day_of_week_Friday',
       'day_of_week_Monday', 'day_of_week_Saturday', 'day_of_week_Sunday',
       'day_of_week_Thursday', 'day_of_week_Tuesday', 'day_of_week_Wednesday'],
      dtype='object', length=153)

In [ ]:
## Next we will try to turn all numerical data into categories and make a prediction with Randomforestclassifierb

In [34]:
flights_df_s = flights_df[arr_delay].copy()

<bound method NDFrame.head of 0           0
1           0
2           0
3          20
4           5
           ..
1559040    40
1559041    40
1559042     5
1559043     0
1559044    70
Name: arr_delay, Length: 1559045, dtype: int64>

In [86]:
# Separate numeric columns and encoded columns

encoded_columns = df_encoded.drop(columns=numerical_columns)

In [87]:
# Scale the numeric columns
scaler = StandardScaler()
scaled_numeric_data = scaler.fit_transform(df_encoded[numerical_columns])

# Create a DataFrame for the scaled numeric data
df_scaled_numeric = pd.DataFrame(scaled_numeric_data, columns=numerical_columns)

# Rejoin the scaled numeric data with the encoded columns
df_scaled_encoded = pd.concat([df_scaled_numeric, encoded_columns], axis=1)

In [88]:
# Linear regression model 

# Separate the input features (X) and the target variable (y)
X = df_scaled_encoded

y = flights_df_sample['arr_delay']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
model = LinearRegression()

# Train the model using the training data
model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = model.predict(X_test)

# Evaluate the model
score = model.score(X_test, y_test)

# Print the coefficient and intercept of the linear regression model
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)
print('R^2 Score:', score)

Coefficients: [ 3.80008193e+01 -3.85794029e+01 -2.31536916e+12 -7.55923083e+12
 -2.80552116e+11  2.76223579e+00  5.75691489e+00  3.44114876e+00
 -1.62263761e+12  9.42743735e+11 -1.19937102e+12  4.48498612e+12
  4.01251733e+12  3.50020085e+12  4.51140029e+12  3.67001259e+12
  4.50432689e+12  4.30109343e+12  3.51601773e+12  3.89798065e+12
  4.28482167e+12  3.80832656e+12  3.73557609e+12  1.39711853e+12
  3.43950465e+13  1.80672202e+13  3.73977129e+13  3.13404899e+13
  2.10754339e+13  4.28559699e+12  9.14425955e+12  8.14881366e+12
  4.24211206e+13  3.69899677e+13  3.83697378e+13  2.65980118e+13
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  1.64283995e+12  1.64283995e+12  1.64283995e+12  1.64283995e+12
  4.8641942